In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import APSIMGraphHelpers as AGH
import GraphHelpers as GH
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.dates as mdates
import MathsUtilities as MUte
import shlex # package to construct the git command to subprocess format
import subprocess 
import xmltodict, json
import sqlite3
from scipy.optimize import basinhopping
from scipy.optimize import dual_annealing
from skopt import gp_minimize
%matplotlib inline

In [8]:
con = sqlite3.connect(r'C:\GitHubRepos\ApsimX\Tests\Validation\Wheat\Wheat.db')
DailyReport = pd.read_sql("Select * from DailyReport",con)

In [19]:
Simulations = pd.read_sql("Select * from _Simulations",con)
Simulations.set_index('ID',inplace=True)

In [9]:
Observed = pd.read_sql("Select * from Observed",con).dropna(axis=1,how='all')
Observed.sort_index(inplace=True)
Observed.sort_index(inplace=True,axis=1)

In [10]:
Factors = pd.read_sql("Select * from _Factors",
                    con)
Factors.set_index('SimulationID',drop=False,inplace=True)
Factors = Factors.sort_values(by=['FactorName']).drop_duplicates()
Factors.sort_index(inplace=True)
Factors.sort_index(inplace=True,axis=1)

In [20]:
Simulations

,Name,FolderName
ID,,
1,APS26NRate0WaterDry,SE Queensland
2,APS26NRate40WaterDry,SE Queensland
3,APS26NRate40WaterWet,SE Queensland
4,APS26NRate0WaterWet,SE Queensland
5,APS26NRate80WaterWet,SE Queensland
...,...,...
7276,YanYean2020TOS6CvWhistler,None
7277,YanYean2020TOS6CvWills,None
7278,YanYean2020TOS6CvWyalkatchem,None


In [11]:
list(Observed.columns)

['([Wheat].Leaf.Transpiration + [Soil].SoilWater.Es + [MicroClimate].PrecipitationInterception)',
 'CheckpointID',
 'Clock.Today',
 'NDVIModel.Script.NDVI',
 'Simulation.Name',
 'SimulationID',
 'Soil.Water.Volumetric(1)',
 'Soil.Water.Volumetric(10)',
 'Soil.Water.Volumetric(2)',
 'Soil.Water.Volumetric(3)',
 'Soil.Water.Volumetric(4)',
 'Soil.Water.Volumetric(5)',
 'Soil.Water.Volumetric(6)',
 'Soil.Water.Volumetric(7)',
 'Soil.Water.Volumetric(8)',
 'Soil.Water.Volumetric(9)',
 'Wheat.AboveGround.N',
 'Wheat.AboveGround.Wt',
 'Wheat.Ear.N',
 'Wheat.Ear.Nconc',
 'Wheat.Ear.Wt',
 'Wheat.Grain.FWt',
 'Wheat.Grain.FWt15',
 'Wheat.Grain.GrainSize',
 'Wheat.Grain.N',
 'Wheat.Grain.NConc',
 'Wheat.Grain.Number',
 'Wheat.Grain.Protein',
 'Wheat.Grain.Size',
 'Wheat.Grain.Wt',
 'Wheat.Leaf.AppearedCohortNo',
 'Wheat.Leaf.CohortMaxSize(1)',
 'Wheat.Leaf.CohortMaxSize(10)',
 'Wheat.Leaf.CohortMaxSize(11)',
 'Wheat.Leaf.CohortMaxSize(12)',
 'Wheat.Leaf.CohortMaxSize(13)',
 'Wheat.Leaf.CohortMax

In [16]:
list(DailyReport.columns)

['CheckpointID',
 'SimulationID',
 'Experiment',
 'FolderName',
 'NRate',
 'Water',
 'Zone',
 'Simulation.Name',
 'Clock.Today',
 'Soil.Water.Volumetric(1)',
 'Soil.Water.Volumetric(2)',
 'Soil.Water.Volumetric(3)',
 'Soil.Water.Volumetric(4)',
 'Soil.Water.Volumetric(5)',
 'Soil.Water.Volumetric(6)',
 'Soil.Water.Volumetric(7)',
 'Soil.Water.Volumetric(8)',
 'Soil.Water.Volumetric(9)',
 'sum(Soil.Water.MM)',
 'sum(Soil.NO3.kgha)',
 'IWeather.Radn',
 'IWeather.MeanT',
 'Wheat.AboveGround.N',
 'Wheat.AboveGround.Wt',
 'Wheat.AboveGroundLive.N',
 'Wheat.AboveGroundLive.Wt',
 'Wheat.AboveGroundDead.N',
 'Wheat.AboveGroundDead.Wt',
 'Wheat.Ear.N',
 'Wheat.Ear.Nconc',
 'Wheat.Ear.Wt',
 'Wheat.Grain.N',
 'Wheat.Grain.NConc',
 'Wheat.Grain.Number',
 'Wheat.Grain.Size',
 'Wheat.Grain.Wt',
 'Wheat.Leaf.AppearedCohortNo',
 'Wheat.Leaf.CoverGreen',
 'Wheat.Leaf.CoverTotal',
 'Wheat.Leaf.Dead.N',
 'Wheat.Leaf.Dead.NConc',
 'Wheat.Leaf.Dead.Wt',
 'Wheat.Leaf.DeadCohortNo',
 'Wheat.Leaf.SenescingCoh

In [17]:
Factors

,CheckpointID,ExperimentName,FactorName,FactorValue,FolderName,SimulationID
SimulationID,,,,,,
1,1,APS26,Water,Dry,SE Queensland,1
1,1,APS26,NRate,0,SE Queensland,1
2,1,APS26,NRate,40,SE Queensland,2
2,1,APS26,Water,Dry,SE Queensland,2
3,1,APS26,NRate,40,SE Queensland,3
...,...,...,...,...,...,...
6499,1,YanYean2020,Cv,Axe,NPIField2020,6499
6500,1,CO2TE,CO2,350ppm,CO2AndTranspirationEfficiency,6500
6501,1,YanYean2020,TOS,7,NPIField2020,6501


In [18]:
Observed

,([Wheat].Leaf.Transpiration + [Soil].SoilWater.Es + [MicroClimate].PrecipitationInterception),CheckpointID,Clock.Today,NDVIModel.Script.NDVI,Simulation.Name,SimulationID,Soil.Water.Volumetric(1),Soil.Water.Volumetric(10),Soil.Water.Volumetric(2),Soil.Water.Volumetric(3),...,Wheat.Stem.N,Wheat.Stem.NConc,Wheat.Stem.NonStructural.Wt,Wheat.Stem.Wt,Wheat.Structure.BranchNumber,Wheat.Structure.HaunStageFloralInitiation,Wheat.Structure.LeafTipsAppeared,Wheat.Structure.TotalStemPopn,sum([Soil].Water.MM),sum([Soil].Water.MM[1:7])
0,NaN,1,None,NaN,None,15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1,None,NaN,None,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,None,NaN,None,21,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1,None,NaN,None,25,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1,None,NaN,None,16,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29062,NaN,1,2024-01-29 12:00:00,NaN,None,801,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29063,NaN,1,2024-01-29 12:00:00,NaN,None,804,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29064,NaN,1,2024-01-29 12:00:00,NaN,None,805,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29065,NaN,1,2024-01-29 12:00:00,NaN,None,793,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
for i in Observed.index:
    simulationID = Observed.loc[i,'SimulationID']
    Observed.loc[i,['ExperimentName', 'FactorName', 'FactorValue','FolderName']] = Factors.loc[simulationID,['ExperimentName', 'FactorName', 'FactorValue','FolderName']]

ValueError: Incompatible indexer with Series